In [0]:
import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, AvgPool2D, DepthwiseConv2D
from keras.layers import Flatten, concatenate, Dropout, ReLU, GlobalAvgPool2D, add, Lambda

from keras.models import Model #takes input and output tensor. Builds the model for us 

import keras.backend as K
from time import time
import numpy as np

In [0]:
def toy_model(input_shape, n_channels):
  input = Input(input_shape) #input tensor 
  
  x = Conv2D(64,11,strides=5,padding='valid',activation='relu')(input)
  #We want to know how many kernels we apple to image (64), size of the kernels if square the same, stride number, padding incase you want to retain the same size 'same' or 'valid')
  x = Flatten()(x)
  output = Dense(n_channels, activation='softmax')(x) #outut tensor 
  
  model = Model(input,output)
  return model

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = toy_model(input_shape, n_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 43, 43, 64)        23296     
_________________________________________________________________
flatten_1 (Flatten)          (None, 118336)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              118337000 
Total params: 118,360,296
Trainable params: 118,360,296
Non-trainable params: 0
_________________________________________________________________


In [0]:
def alexnet(input_shape, n_classes):
  input = Input(input_shape) #input tensor 
  x = Conv2D(96,11,strides=4,activation='relu', padding='same')(input)
  x = BatchNormalization()(x)
  x = MaxPool2D(3,strides=2)(x)
  
  #Diger block
  x = Conv2D(256, 5, padding='same', activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPool2D(3,strides=2)(x)
  
  x = Conv2D(384, 3, padding='same', activation='relu')(x)
  x = Conv2D(384, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  
  x = BatchNormalization()(x)
  x = MaxPool2D(3,strides=2)(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  return model
  

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = alexnet(input_shape, n_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 256)       0         
__________

In [0]:
def vgg(input_shape,n_classes):
  input = Input(input_shape)
  x = Conv2D(64,3,activation='relu', padding='same')(input)
  x = Conv2D(64,3,activation='relu', padding='same')(x)
  
  x = MaxPool2D(3,strides=2)(x)
  
  x = Conv2D(128,3,activation='relu', padding='same')(x)
  x = Conv2D(128,3,activation='relu', padding='same')(x)
  
  x = MaxPool2D(3,strides=2)(x)
  
  x = Conv2D(256,3,activation='relu', padding='same')(x)
  x = Conv2D(256,3,activation='relu', padding='same')(x)
  x = Conv2D(256,3,activation='relu', padding='same')(x)
  
  x = MaxPool2D(3,strides=2)(x)
  
  x = Conv2D(512,3,activation='relu', padding='same')(x)
  x = Conv2D(512,3,activation='relu', padding='same')(x)
  x = Conv2D(512,3,activation='relu', padding='same')(x)
  
  x = MaxPool2D(3,strides=2)(x)
  
  x = Conv2D(512,3,activation='relu', padding='same')(x)
  x = Conv2D(512,3,activation='relu', padding='same')(x)
  x = Conv2D(512,3,activation='relu', padding='same')(x)
  
  x = MaxPool2D(3,strides=2)(x)
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  return model
  

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = vgg(input_shape, n_classes)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 111, 111, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 55, 128)       0         
__________

In [0]:
 def googlenet(input_shape, n_classes):
  def incpeption_block(x, f):
    t1 = Conv2D(f[0], 1, activation='relu')(x)
    
    t2 = Conv2D(f[1], 1, activation='relu')(x)
    t2 = Conv2D(f[2], 3, padding='same', activation='relu')(t2)
    
    t3 = Conv2D(f[3], 1, activation='relu')(x)
    t3 = Conv2D(f[4], 5, padding='same', activation='relu')(t3)
    
    t4 = MaxPool2D(3, strides=1, padding='same')(x)
    t4 = Conv2D(f[5], 1, padding='same',activation='relu')(t4)
    
    output = concatenate([t1, t2, t3, t4])
    return output
  
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = Conv2D(64, 1, activation='relu')(x)
  x = Conv2D(192, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = incpeption_block(x, [64, 96, 128, 16, 32, 32])
  x = incpeption_block(x, [128, 128, 192, 32, 96, 64])
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = incpeption_block(x, [192, 96, 208, 16, 48, 64])
  x = incpeption_block(x, [160, 112, 224, 24, 64, 64])
  x = incpeption_block(x, [128, 128, 256, 24, 64, 64])
  x = incpeption_block(x, [112, 144, 288, 32, 64, 64])
  x = incpeption_block(x, [256, 160, 32, 32, 128, 128])
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = incpeption_block(x, [256, 160, 320, 32, 128, 128])
  x = incpeption_block(x, [384, 192, 384, 48, 128, 128])
  
  x = AvgPool2D(7, strides=1)(x)
  x = Dropout(0.4)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = googlenet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_3 (

In [0]:
def mobilenet(input_shape,n_classes):
  def conv_bn_rl(x, f, k=1, s=1, p='same'): #Cunku her block bir batchnorm ve relu takip ediyor 
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  def mobilenet_block(x, f, s=1):
    x = DepthwiseConv2D(3, s, padding='same')(x) #depthwise separable conv 
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = conv_bn_rl(x, f)
    return x
  
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 32, 3, 2)
  x = mobilenet_block(x, 64) #kirmizi blok
  
  x = mobilenet_block(x, 128, 2) #128 stride 2
  x = mobilenet_block(x, 128)  # 128 stride 1
  
  x = mobilenet_block(x, 256, 2)
  x = mobilenet_block(x, 256)
  
  x = mobilenet_block(x, 512, 2)
  for i in range(5):
    x = mobilenet_block(x, 512)
    
  #pdfte aradaki typo
  x = mobilenet_block(x, 1024, 2)
  x = mobilenet_block(x, 1024)
  
  x = AvgPool2D(7)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model
  

In [0]:
repetitions = 10
input = np.random.randn(1, *input_shape)

output = model.predict(input)
start = time()
for _ in range(repetitions):
  output = model.predict(input)
  
print((time() - start) / repetitions)

0.064558506012


In [0]:
def shufflenet(input_shape, n_classes, g=8):
   #g in the paper is number of groups 
  #this is implemnted for 8 groups 
  
  #filters , repetititons her stage kac kere tekrar etmis 
  channels = 384, 769, 1536
  repetitions = 3, 7, 3
  
  def ch_shuffle(x, g):
    #We transpose, permute the dimensions 
    '''
    [1 2 3 4 5 6 7 8  9]
    V  
    V
    [1 2 3
    4 5 6
    7 8 9]
    V
    V
    [1 4 7 
    2 5 8
    3 6 9]
    V
    V
    [1 4 7 2 5 8 3 6 9]
    '''
    _, w, h, ch = K.int_shape(x) #This x is a Tensorflow tensor, we need to change it to Keras
    ch_g = ch // g

    
    def shuffle_op(x): #With Lambda we can change a tf tensor to Keras tensor 
      x = K.reshape(x, [-1, w, h, ch_g, g])
      x = K.permute_dimensions(x, [0, 1, 2, 4, 3])
      x = K.reshape(x, [-1, w, h, ch])
      return x
    
    x = Lambda(shuffle_op)(x)
    return x
    
  
  def gconv(tensor, ch, g):
    _, _, _, in_ch = K.int_shape(tensor)
    ch_g = in_ch // g
    out_ch = ch // g
    group = []
    
    for i in range(g):
      #x = tensor[:, :, :, i*ch_g:(i+1)*ch_g]  bunu yapmak istiyoruz ama bunuu yapamayiz. Her lambda icin bu islemi uygula sonucu x e at demek. import Lambda 
      x = Lambda(lambda x: x[:, :, :, i*ch_g: (i+1)*ch_g])(tensor) #4 to 8, 8 to 12 etc..
      x = Conv2D(out_ch, 1)(x)
      group.append(x)
    
    x = concatenate(group)
    return x
  
    #asagida her shufflein kullandigi her block nasil tanilaniyor 

  def shufflenet_block(tensor, ch, s, g):
     #"the bottleneck ratio1 is also 1 : 4".
    x = gconv(tensor, ch // 4, g)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = ch_shuffle(x, g)
    x = DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = BatchNormalization()(x)
    x = gconv(x, ch if s==1 else ch-K.int_shape(tensor)[-1], g)
    x = BatchNormalization()(x)
    
    if s == 1:
      x = add([tensor, x])
    else:
      avg = AvgPool2D(3, strides=2, padding='same')(tensor)
      x = concatenate([avg, x])
    
    output = ReLU()(x)
    return output
  
  #Her stagei nasil yapiyoruz. 1 kere stride 2, kalani stride 1 
  def stage(x, ch, r, g):
    x = shufflenet_block(x, ch, 2, g)
    
    for i in range(r):
      x = shufflenet_block(x, ch, 1, g)
      
    return x
  
  input = Input(input_shape)
  
  x = Conv2D(24, 3, strides=2, padding='same')(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for ch, r in zip(channels, repetitions):
    x = stage(x, ch, r, g)
    
  x = GlobalAvgPool2D()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model
  

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = shufflenet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 24) 672         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 24) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 112, 112, 24) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [0]:
def resnet(input_shape, n_classes):
  #PDF kirmizi kutuda x6 demesi 1 conv x5 identitiy block 
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  def conv_block(tensor, f1, f2, s):
    x = conv_bn_rl(tensor, f1, s=s)
    x = conv_bn_rl(x, f1, 3)
    x = Conv2D(f2, 1)(x)
    x = BatchNormalization()(x)

    shortcut = Conv2D(f2, 1, strides=s, padding='same')(tensor)
    shortcut = BatchNormalization()(shortcut)

    x = add([shortcut, x])
    output = ReLU()(x)

    return output

  
  def identity_block(tensor, f1, f2):
    x = conv_bn_rl(tensor, f1)
    x = conv_bn_rl(x, f1, 3)
    x = Conv2D(f2, 1)(x)
    x = BatchNormalization()(x)
    
    x = add([tensor, x])
    output = ReLU()(x)
    
    return output
    
  
  
  def resnet_block(x, f1, f2, r, s=2):
    x = conv_block(x, f1, f2, s)
    
    for _ in range(r-1):
      x = identity_block(x, f1, f2)
    
    return x

  input = Input(input_shape)
  #size of the tensor smaller so we need to adjust input tensor 
  #We apply a conv layer just to change the size
  x = conv_bn_rl(input, 64, 7, 2)
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = resnet_block(x, 64, 256, 3, 1)
  x = resnet_block(x, 128, 512, 4)
  x = resnet_block(x, 256, 1024, 6)
  x = resnet_block(x, 512, 2048, 3,2)

  x = AvgPool2D(7, strides=1)(x)
  x = Flatten()(x)
    
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model

    
  

In [0]:
input_shape = 224, 224, 3
n_classes = 1000

K.clear_session()
model = resnet(input_shape, n_classes)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [0]:
#conv means batchnorm-relu-conv in pfd 
def densenet(input_shape, n_classes, k=32):
  repetitions = 6, 12, 48, 32
  #conv means batchnorm-relu-conv in pfd  this block 
  def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x
  
  #k is the number of filters for all the convolution layers 
  def dense_block(tensor, k):
    x = bn_rl_conv(tensor, 4*k)
    x = bn_rl_conv(x, k, 3)
    output = concatenate([tensor, x])
    return output
  
  #Block that shrinks the size 
  def transition(x):
    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)  #whatever the number of channels, we take the half of it 
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x
  
  
  def dense_blocks(x, r, k):
    for _ in range(r):
      x = dense_block(x, k)
    return x
  
  
  input = Input(input_shape)
  #basta conv ve maxpool la basladik 
  x = Conv2D(64, 7, strides=2, padding='same')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  for r in repetitions:
    d = dense_blocks(x, r, k)
    x = transition(d)
  
  x = AvgPool2D(7, strides=1)(d)
  x = Flatten()(x)
  
  output = Dense(n_classes, activation='softmax')(x)
  model = Model(input, output)
  
  return model